In [16]:
from dataset1 import *
import numpy as np
from scipy.linalg import solve_triangular
from numpy.linalg import eigh
import torch
from hadamard_transform import hadamard_transform
import time
import pandas as pd
import seaborn as sns
#test

## Define functions

In [17]:

def getError(A,B):
    return np.linalg.norm(A-B,ord='nuc')/np.linalg.norm(A,ord='nuc')

def fjlt(A, D, indices, direction):
    """
    Applies the Fast Johnson-Lindenstrauss Transform (FJLT) to a square matrix.

    Parameters:
    - A: Input square matrix (NumPy array of size n x m).
    - D: Diagonal vector of +1 and -1 (NumPy array of size n).
    - indices: Indices of the selected rows or columns for subsampling (NumPy array of size l).
    - direction: 'left' to apply FJLT from the left (on rows) or 'right' for the transpose case (on columns).

    Returns:
    - A_proj: Transformed matrix after applying FJLT and subsampling.
    """
    n, m = A.shape
    l = indices.shape[0]  # Sketch size
    if direction == 'left':  # FJLT applied to the rows of A
        assert len(D) == n, "The length of D must match the number of rows in A."
        assert (n & (n - 1) == 0), "The number of rows in A must be a power of 2."
        # Multiply each row of A by the corresponding element in D
        OmegaA = A * D[:, np.newaxis]
        # Apply the Fast Hadamard Transform to the rows
        OmegaA = hadamard_transform(torch.tensor(OmegaA.T)).numpy().T
        # Subsample the rows and scale
        OmegaA = OmegaA[indices, :] * np.sqrt(n / l)
        return np.ascontiguousarray(OmegaA)
    elif direction == 'right':  # FJLT applied to the columns of A
        # Multiply each column of A by the corresponding element in D
        AOmega = A * D[np.newaxis, :]
        # Apply the Fast Hadamard Transform to the columns
        AOmega = hadamard_transform(torch.tensor(AOmega)).numpy()
        # Subsample the columns and scale
        AOmega = AOmega[:, indices] * np.sqrt(n / l)
        return np.ascontiguousarray(AOmega)

def getLowRank(A, rank, sketch_dim, sketch_method='gaussian', good_conditionned=True, print_times=False):
    """
    Compute a low-rank approximation of the matrix A using sketching, 
    eigenvalue decomposition, and truncation. Optionally prints the time taken for each major operation.

    Parameters:
        A (numpy.ndarray): Original matrix (size m x n).
        rank (int): Desired rank for the approximation.
        sketch_dim (int): Dimension of the sketching subspace.
        sketch_method (str): Method used for sketching. Options are:
                             - 'gaussian': Uses a Gaussian random matrix for sketching.
                             - 'fjlt': Uses the Fast Johnson-Lindenstrauss Transform.
        good_conditionned (bool): If True, uses Cholesky decomposition for better conditioning. 
                                  Otherwise, uses eigenvalue decomposition.
        print_times (bool): If True, prints the time taken for each major operation.

    Returns:
        A_nyst_k (numpy.ndarray): Low-rank approximation of A (size m x m).
    """
    m = A.shape[0]
    B = None
    C = None

    start_time = time.time()

    # Apply sketching based on the selected method
    if sketch_method == 'fjlt':  # Use Fast Johnson-Lindenstrauss Transform for sketching
        D = np.random.choice([1, -1], size=m)
        indices = np.random.choice(m, size=sketch_dim, replace=False)
        sketch_start = time.time()
        C = fjlt(A, D, indices, direction='right')
        B = fjlt(C, D, indices, direction='left')
        if print_times:
            print(f"FJLT sketching completed in {time.time() - sketch_start:.6f} seconds.")

    elif sketch_method == 'gaussian':  # Use Gaussian random matrix for sketching
        Omega = np.random.normal(size=(m, sketch_dim))
        sketch_start = time.time()
        C = A @ Omega
        B = Omega.T @ C
        if print_times:
            print(f"Gaussian sketching completed in {time.time() - sketch_start:.6f} seconds.")
    else:
        raise ValueError("Méthode de sketching non reconnue. Choisissez 'gaussian' ou 'fjlt'.")

    if good_conditionned:
        # Use Cholesky decomposition
        cholesky_start = time.time()
        L = np.linalg.cholesky(B)
        if print_times:
            print(f"Cholesky decomposition completed in {time.time() - cholesky_start:.6f} seconds.")
        
        triangular_start = time.time()
        Z = solve_triangular(L, C.T, lower=True).T
        if print_times:
            print(f"Triangular solve completed in {time.time() - triangular_start:.6f} seconds.")
    else:
        # Use eigenvalue decomposition
        eigh_start = time.time()
        eig_v, V = eigh(B)
        if print_times:
            print(f"Eigenvalue decomposition completed in {time.time() - eigh_start:.6f} seconds.")
        
        truncate_start = time.time()
        truncate = eig_v > 0
        eig_v = eig_v[truncate]
        V = V[:, truncate]
        Z = C @ (V * (1 / np.sqrt(eig_v)))
        if print_times:
            print(f"Eigenvalue truncation and normalization completed in {time.time() - truncate_start:.6f} seconds.")

    qr_start = time.time()
    Q, R = np.linalg.qr(Z)
    if print_times:
        print(f"QR decomposition completed in {time.time() - qr_start:.6f} seconds.")

    svd_start = time.time()
    U, Sigma, _ = np.linalg.svd(R)
    if print_times:
        print(f"SVD completed in {time.time() - svd_start:.6f} seconds.")

    approx_start = time.time()
    U_hat = Q @ U[:, :rank]
    A_nyst_k = (U_hat * Sigma[:rank]**2) @ U_hat.T
    if print_times:
        print(f"Low-rank approximation construction completed in {time.time() - approx_start:.6f} seconds.")

    total_time = time.time() - start_time
    if print_times:
        print(f"Total computation time: {total_time:.6f} seconds.")
    error = getError(A,A_nyst_k)
    print(f"Error :{error}".format(error))

    return A_nyst_k, error, 

## Tests

In [20]:
n = 2048
A = getExpMatrix(n,R=10)

In [21]:
getLowRank(A,rank=128,sketch_dim=128,sketch_method='gaussian',good_conditionned=False,print_times=True)

Gaussian sketching completed in 0.022853 seconds.
Eigenvalue decomposition completed in 0.009401 seconds.
Eigenvalue truncation and normalization completed in 0.002759 seconds.
QR decomposition completed in 0.027138 seconds.
SVD completed in 0.005126 seconds.
Low-rank approximation construction completed in 0.032776 seconds.
Total computation time: 0.115525 seconds.
Error :8.329604176432034e-15


(array([[ 1.00000000e+00, -1.16920362e-15,  2.49800181e-16, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-1.11716192e-15,  1.00000000e+00, -2.77555756e-17, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 2.49800181e-16, -5.55111512e-17,  1.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]]),
 8.329604176432034e-15)

In [27]:
getLowRank(A,rank=128,sketch_dim=128,sketch_method='fjlt',good_conditionned=False,print_times=True)

FJLT sketching completed in 0.151482 seconds.
Eigenvalue decomposition completed in 0.007486 seconds.
Eigenvalue truncation and normalization completed in 0.001138 seconds.
QR decomposition completed in 0.031378 seconds.
SVD completed in 0.005304 seconds.
Low-rank approximation construction completed in 0.012530 seconds.
Total computation time: 0.210103 seconds.


KeyboardInterrupt: 

## Section 2

We'll compute the error with respect to three matrices. 
1. polynomial decay (dimension 1000 * 1000) locates: 
2. exp decay locates:
3. ../Dataset/A_3_dataset_2_mnist.npy 
    denotes the matrix obtained from MNIST, with c=100
4. 

In [23]:
# Loading matrices

loadMatrix_A_3 = np.load("../Dataset/A_3_dataset_2_mnist.npy")
print("matrix A_3")
print(loadMatrix_A_3.shape)
print(loadMatrix_A_3[:10,:5])

loadMatrix_A_1 = np.load("../Dataset/A_1_polyDecayMatrix.npy")
loadMatrix_A_2 = np.load("../Dataset/A_2_expDecayMatrix.npy")

print("matrix A_1")
print(loadMatrix_A_1.shape)
print(loadMatrix_A_1[:10,:5])

print("matrix A_2")
print(loadMatrix_A_2.shape)
print(loadMatrix_A_2[:10,:5])

matrix A_3
(1000, 1000)
[[1.         0.99127504 0.98824289 0.99001237 0.98907651]
 [0.99127504 1.         0.98715919 0.98829865 0.98930957]
 [0.98824289 0.98715919 1.         0.9898814  0.98994457]
 [0.99001237 0.98829865 0.9898814  1.         0.99126262]
 [0.98907651 0.98930957 0.98994457 0.99126262 1.        ]
 [0.98979753 0.98903161 0.98938555 0.99024257 0.99195122]
 [0.99100754 0.98756737 0.98972827 0.991512   0.9925866 ]
 [0.99278471 0.98967594 0.98934556 0.98962114 0.98935745]
 [0.99136177 0.98870132 0.99122707 0.99290202 0.9940263 ]
 [0.98989935 0.98824599 0.99277739 0.99119833 0.99174104]]
matrix A_1
(1000, 1000)
[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
matrix A_2
(1000, 1000)
[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0.

In [24]:
from tqdm import tqdm  # For progress tracking

def collect_low_rank_errors(A, max_rank=128, sketch_dims=range(10, 101, 10), is_good_conditionned=False, sketch_method='gaussian', output_file='Section_2_test_1_for_loadMatrixA_1.xlsx'):
    """
    Computes errors for various ranks and sketch dimensions using the `getLowRank` function.

    Parameters:
    - A (np.ndarray): Input matrix of size, for example 1000x1000.
    - max_rank (int): Maximum rank to test (0 <= rank <= max_rank).
    - sketch_dims (iterable): Values of sketch dimensions to test.
    - sketch_method (str): Method for sketching ('gaussian' or other supported by `getLowRank`).
    - output_file (str): Path to the file where results are saved (.csv or .xlsx).
    
    Saves:
    - A table with columns: "Approximation Rank", "Sketch Dimension", "Error Value",
      clearly labeled for reuse and analysis.
    """
    # Ensure the matrix is square and 1000x1000
    # assert A.shape == (1000, 1000), "Matrix A must be of size 1000x1000."
    
    # Prepare a list to store results
    results = []
    
    # Iterate through all rank and sketch_dim combinations
    for rank in tqdm(range(max_rank + 1), desc="Ranks Progress", unit="rank"):
        for sketch_dim in sketch_dims:
            # Compute the error
            error = getLowRank(A, rank, sketch_dim, sketch_method, good_conditionned = is_good_conditionned)
            # Append results with clear column names
            results.append({
                'Approximation Rank': rank,
                'Sketch Dimension': sketch_dim,
                'Error Value': error,
                'Sketch Method': sketch_method  # Added for better traceability
            })
    
    # Convert to DataFrame for easy export
    df = pd.DataFrame(results)
    
    # Save results to a human-readable format
    if output_file.endswith('.csv'):
        df.to_csv(output_file, index=False)
    elif output_file.endswith('.xlsx'):
        df.to_excel(output_file, index=False, engine='openpyxl')
    else:
        raise ValueError("Output file must be either .csv or .xlsx")
    
    print(f"Results successfully saved to {output_file}.")


In [25]:
collect_low_rank_errors(loadMatrix_A_1)

Ranks Progress:   0%|          | 0/129 [00:00<?, ?rank/s]

Error :1.0
Error :1.0
Error :1.0
Error :1.0
Error :1.0
Error :1.0
Error :1.0
Error :1.0
Error :1.0


Ranks Progress:   1%|          | 1/129 [00:05<12:02,  5.65s/rank]

Error :1.0
Error :0.9482924928005755
Error :0.9457201976509276
Error :0.9437538416677704
Error :0.9425313559274502
Error :0.9415654469594548
Error :0.9413661874204159
Error :0.9413237319266492
Error :0.9407889385174028
Error :0.9407501496845445


Ranks Progress:   2%|▏         | 2/129 [00:11<12:26,  5.88s/rank]

...

Ranks Progress:  99%|█████████▉| 128/129 [13:40<00:06,  6.24s/rank]

Error :0.24619464044160919
Error :0.7122828580427869
Error :0.5630334868065253
Error :0.4459269651348106
Error :0.3901434591965976
Error :0.3416379342466799
Error :0.314695576613568
Error :0.2936365901467728
Error :0.271991786528246
Error :0.2570710718482284


Ranks Progress: 100%|██████████| 129/129 [13:46<00:00,  6.40s/rank]

Error :0.24450886564889032


Results successfully saved to Section_2_test_1_for_loadMatrixA_1.xlsx.


In [26]:
collect_low_rank_errors(A=loadMatrix_A_1, output_file='../Dataset/Section_2_test_2_for_loadMatrixA_1.csv')
collect_low_rank_errors(A=loadMatrix_A_2, output_file='../Dataset/Section_2_test_2_for_loadMatrixA_2.csv')
collect_low_rank_errors(A=loadMatrix_A_3, output_file='../Dataset/Section_2_test_2_for_loadMatrixA_3.csv')

Ranks Progress:   0%|          | 0/129 [00:00<?, ?rank/s]

Error :1.0
Error :1.0
Error :1.0
Error :1.0
Error :1.0
Error :1.0
Error :1.0
Error :1.0
Error :1.0


Ranks Progress:   1%|          | 1/129 [00:05<12:32,  5.88s/rank]

Error :1.0
Error :0.9504192649843474
Error :0.9440374499488048
Error :0.9427072264467282
Error :0.9416234434056384
Error :0.9417250929357813
Error :0.941264925379735
Error :0.9408831422630048
Error :0.9410011178258328
Error :0.9407829339613573


Ranks Progress:   2%|▏         | 2/129 [00:12<13:20,  6.30s/rank]

...

Ranks Progress:   0%|          | 0/129 [00:00<?, ?rank/s]

Error :1.0
Error :1.0
Error :1.0
Error :1.0
Error :1.0
Error :1.0


Ranks Progress:   0%|          | 0/129 [00:03<?, ?rank/s]

Error :1.0


LinAlgError: Matrix is not positive definite